<a href="https://colab.research.google.com/github/Tituswan05/Hilti-WhatsApp-Demo1.0/blob/main/Backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance ta
!pip install --upgrade --force-reinstall --no-cache-dir "numpy==2.0.2" "pandas==2.2.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 145.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 196.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 288.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 345.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 257.5 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2025.2
    Uninstalling tzdata-2025.2:
      Successfully uninstalled tzdata-2025.2
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1.17.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import re
import io
import numpy as np
import pandas as pd
import yfinance as yf
import requests
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

START, END            = "2020-01-01", "2025-10-28"
LOOKBACK_DAYS         = 400
INIT_HKD              = 130_000
MONTHLY_HKD           = 3_500
HKD2USD               = 7.78  # Updated based on September 26, 2025 rate
RISK_PCT              = 0.1
ATR_RATIO             = 1
ATR_TSL_MULT          = 5.0
ATR_SL_MULT           = 3.0
VOL_MULT              = 1.2
BREAKEVEN_DAYS        = 5
BREAKEVEN_PCT         = 0.0
RSI_LOW               = 40
ROLL_DAYS             = 126
ETF_LIST              = ["QQQ"]
UNIVERSE_SOURCE       = ("https://en.wikipedia.org/wiki/Nasdaq-100", "Ticker")
HEADERS               = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/115.0.0.0 Safari/537.36"
    )
}
COMMISSION            = 0.0049
PLATFORM              = 0.0050
MIN_FEE               = 1.99
SLIPPAGE_MEAN_PCT     = 0.0005
SLIPPAGE_STD_PCT      = 0.0002
MAX_POSITIONS         = 7
MAX_PER_TICKER_PCT    = 0.5

def calc_fee(qty):
    return max(qty * (COMMISSION + PLATFORM), MIN_FEE)

def apply_slippage(price, qty, vol):
    if vol == 0:
        vol = 1
    impact = 0.01 * np.sqrt(qty / vol)  # Quadratic market impact term
    rnd = np.random.normal(loc=SLIPPAGE_MEAN_PCT + impact, scale=SLIPPAGE_STD_PCT)
    return price * (1 + rnd)

def clean_tickers(raw):
    return [t for t in raw if re.fullmatch(r"[A-Z\.]{1,5}", t)]

def fetch_tickers():
    try:
        resp = requests.get(UNIVERSE_SOURCE[0], headers=HEADERS, timeout=10)
        resp.raise_for_status()
        tables = pd.read_html(io.StringIO(resp.text))
    except Exception as e:
        print(f"Warning: 無法取得成分股列表, 原因: {e}")
        return ETF_LIST.copy()
    raw = []
    for tbl in tables:
        cols = [c.lower() for c in tbl.columns.astype(str)]
        if "ticker" in cols or "symbol" in cols:
            raw.extend(tbl[UNIVERSE_SOURCE[1]].astype(str).str.upper())
            break
    current_tickers = list(dict.fromkeys(clean_tickers(raw) + ETF_LIST))
    historical_tickers = [
        'CDW', 'CCEP', 'DASH', 'MDB', 'ROP', 'SPLK', 'TTWO', 'ON', 'GEHC',
        'ALGN', 'EBAY', 'ENPH', 'JD', 'LCID', 'ZM', 'SGEN', 'ATVI', 'RIVN', 'FI',
        'PLTR', 'MSTR', 'AXON', 'APP', 'SMCI', 'ARM', 'LIN',
        'ILMN', 'MRNA', 'DLTR', 'WBA', 'SIRI',
        'TRI', 'SHOP',
        'ANSS',
        # Additional historical from 2020-2024 removals
        'BMRN', 'CTXS', 'EXPE', 'LBTYA', 'LBTYK', 'ULTA', 'WDC', 'NTAP', 'UAL', 'WTW', 'AAL',
        'FOXA', 'FOX', 'CERN', 'CHKP', 'TCOM', 'INCY', 'VRSN', 'SWKS', 'BIDU', 'MTCH', 'DOCU', 'NTES'
    ]
    all_tickers = list(dict.fromkeys(current_tickers + historical_tickers))
    return all_tickers

def flatten(df):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    return df

def rsi(close, window=14):
    delta = close.diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    ema_up = up.ewm(com=window-1, min_periods=window).mean()
    ema_down = down.ewm(com=window-1, min_periods=window).mean()
    rs = ema_up / ema_down
    return 100 - 100 / (1 + rs)

def atr(high, low, close, window=14):
    tr = pd.concat([high - low, abs(high - close.shift()), abs(low - close.shift())], axis=1).max(axis=1)
    return tr.rolling(window=window).mean()

def macd(close, fast=12, slow=26, signal=9):
    ema_fast = close.ewm(span=fast, adjust=False).mean()
    ema_slow = close.ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    histogram = macd_line - signal_line
    return macd_line, signal_line, histogram

def add_indicators(df):
    df = flatten(df)
    df["MA100"]       = df["Close"].rolling(100, min_periods=50).mean()
    df["MA100_slope"] = df["MA100"].diff(5)
    df["Vol20"]       = df["Volume"].rolling(20, min_periods=10).mean()
    df["ATR14"]       = atr(df["High"], df["Low"], df["Close"])
    df["RSI14"]       = rsi(df["Close"])
    df["MACD"], df["MACD_Signal"], _ = macd(df["Close"])
    df["Up3Pct"]      = (df["Close"] / df["Close"].shift(3) - 1) >= 0.005
    return df.dropna()

def simple_filter(r):
    return (
        r["Close"] > r["MA100"] and
        r["MA100_slope"] > 0 and
        r["Up3Pct"] and
        r["Volume"] > r["Vol20"] * VOL_MULT and
        r["ATR14"]/r["Close"] < ATR_RATIO and
        RSI_LOW <= r["RSI14"] and
        r["MACD"] > r["MACD_Signal"]  # Added multi-factor enhancement
    )

def prepare_universe():
    lookback = (pd.to_datetime(START) - pd.Timedelta(days=LOOKBACK_DAYS)).strftime("%Y-%m-%d")
    uni = {}
    total_buy_signals = 0
    for tk in fetch_tickers():
        try:
            df = yf.download(tk, start=lookback, end=END, progress=False, auto_adjust=True)
        except Exception as e:
            print(f"Warning: Failed to download {tk}, reason: {e}")
            continue
        if df.empty: continue
        df = add_indicators(df)
        df = df[df.index >= pd.to_datetime(START)]
        if df.empty: continue
        df["Buy"] = df.apply(simple_filter, axis=1)
        total_buy_signals += df["Buy"].sum()
        uni[tk] = df
    print(f"總買入訊號數量（所有股票所有日子）：{total_buy_signals}")
    return uni

def run_backtest(uni):
    dates = sorted({d for df in uni.values() for d in df.index})
    cash = INIT_HKD / HKD2USD
    pos, trades = {}, []
    equity, edates = [], []
    last_m, dep = None, 0
    qqq = yf.download("QQQ", start=START, end=END, progress=False, auto_adjust=True)
    qqq = flatten(qqq)
    qqq["MA100"] = qqq["Close"].rolling(100, min_periods=50).mean()
    for dt in dates:
        if last_m != dt.month:
            cash += MONTHLY_HKD / HKD2USD
            dep += 1
            last_m = dt.month
        if dt in qqq.index:
            q_close = qqq.at[dt, "Close"]
            q_ma100 = qqq.at[dt, "MA100"]
            m_ok = q_close > q_ma100
        else:
            m_ok = True
        to_close = []
        for sym, p in pos.items():
            if dt not in uni[sym].index: continue
            r = uni[sym].loc[dt]
            p["high"] = max(p["high"], r["High"])
            if r["Close"] <= p["high"] - ATR_TSL_MULT * r["ATR14"]:
                reason = "TSL"
            elif r["Close"] >= p["entry_p"] * 1.50:
                reason = "TP"
            elif r["Close"] <= p["entry_p"] - ATR_SL_MULT * r["ATR14"]:
                reason = "SL"
            elif (dt - p["entry_d"]).days >= BREAKEVEN_DAYS and r["Close"] <= p["entry_p"]:
                reason = "BE"
            else:
                continue
            exec_p = apply_slippage(r["Close"], p["qty"], r["Vol20"])
            fee = calc_fee(p["qty"])
            profit_usd = (exec_p - p["entry_p"]) * p["qty"] - fee
            cash += p["qty"] * exec_p - fee
            trades.append({
                "ticker": sym,
                "entry_date": p["entry_d"],
                "entry_price": p["entry_p"],
                "exit_date": dt,
                "exit_price": exec_p,
                "profit_pct": ((exec_p - p["entry_p"]) * p["qty"] - fee) /
                              (p["entry_p"] * p["qty"]),
                "profit_usd": profit_usd,
                "reason": reason
            })
            to_close.append(sym)
        for s in to_close:
            del pos[s]
        if m_ok:
            candidates = []
            for sym, df in uni.items():
                if sym in pos or dt not in df.index:
                    continue
                row = df.loc[dt]
                if not row["Buy"]:
                    continue
                if len(df.loc[:dt, "Close"]) >= 5:
                    price_change = (row["Close"] / df.loc[:dt, "Close"].shift(5).iloc[-1] - 1) * 100
                    momentum_score = row["RSI14"] + price_change
                else:
                    momentum_score = row["RSI14"]
                candidates.append((sym, row, momentum_score, df))
            print(f"日期 {dt}: 買入候選人數量 {len(candidates)}")
            candidates.sort(key=lambda x: x[2], reverse=True)
            for sym, row, _, df in candidates[:MAX_POSITIONS]:
                if len(pos) >= MAX_POSITIONS:
                    break
                alloc = cash * MAX_PER_TICKER_PCT
                if row["ATR14"] > 0:
                    risk_qty = int((cash * RISK_PCT) / (ATR_SL_MULT * row["ATR14"]))
                    alloc_qty = int(alloc / row["Close"])
                    qty = min(risk_qty, alloc_qty)
                    if qty == 0 and alloc >= row["Close"]:
                        qty = 1
                else:
                    qty = 0
                cost = qty * row["Close"]
                if qty > 0 and cost <= cash:
                    exec_p = apply_slippage(row["Close"], qty, row["Vol20"])
                    fee = calc_fee(qty)
                    cash -= qty * exec_p + fee
                    pos[sym] = {"entry_p": exec_p, "qty": qty, "high": exec_p, "entry_d": dt}
                    print(f"買入 {sym} 于 {dt}，數量 {qty}，成本 {cost:.2f}")
                else:
                    if len(candidates) > 0:
                        print(f"跳過 {sym} 于 {dt}：qty={qty}, cost={cost:.2f}, alloc={alloc:.2f}, cash={cash:.2f}")
        equity.append(cash + sum(p["qty"] * uni[s]["Close"].loc[dt] for s, p in pos.items() if dt in uni[s].index))
        edates.append(dt)
    last_dt = edates[-1]
    for sym, p in list(pos.items()):
        if last_dt not in uni[sym].index: continue
        r = uni[sym].loc[last_dt]
        exec_p = apply_slippage(r["Close"], p["qty"], r["Vol20"])
        fee = calc_fee(p["qty"])
        profit_usd = (exec_p - p["entry_p"]) * p["qty"] - fee
        cash += p["qty"] * exec_p - fee
        trades.append({
            "ticker": sym,
            "entry_date": p["entry_d"],
            "entry_price": p["entry_p"],
            "exit_date": last_dt,
            "exit_price": exec_p,
            "profit_pct": ((exec_p - p["entry_p"]) * p["qty"] - fee) /
                          (p["entry_p"] * p["qty"]),
            "profit_usd": profit_usd,
            "reason": "EOD"
        })
        del pos[sym]
    equity[-1] = cash
    return pd.Series(equity, index=edates), trades, dep

if __name__ == "__main__":
    uni = prepare_universe()
    eq, trades, dep = run_backtest(uni)
    injected = INIT_HKD + dep * MONTHLY_HKD
    final   = eq.iloc[-1] * HKD2USD
    net     = final - injected
    win_rate= pd.Series([t["profit_pct"] for t in trades]).gt(0).mean() if trades else 0.0
    ret     = (eq.iloc[-1] - eq.iloc[0]) / eq.iloc[0] if eq.iloc[0] != 0 else 0.0
    dd      = (eq.cummax() - eq).max() / eq.cummax().max() if eq.cummax().max() != 0 else 0.0
    roll    = eq.pct_change(periods=ROLL_DAYS).dropna()
    w6, b6  = roll.min() if not roll.empty else 0.0, roll.max() if not roll.empty else 0.0
    # Added metrics
    years = (eq.index[-1] - eq.index[0]).days / 365.25
    ann_ret = (1 + ret) ** (1 / years) - 1 if years > 0 else 0.0
    daily_returns = eq.pct_change().dropna()
    sharpe = daily_returns.mean() / daily_returns.std() * np.sqrt(252) if daily_returns.std() != 0 else 0.0
    print(f"總交易筆數：{len(trades)}")
    print(f"勝率：{win_rate:.2%}")
    print(f"真實總報酬：{ret:.2%}，最大回撤：{dd:.2%}")
    print(f"年化報酬：{ann_ret:.2%}，Sharpe比率：{sharpe:.2f}")
    print(f"投入本金：HK$ {injected:,.0f}")
    print(f"期末本金：約 HK$ {final:,.0f}（淨利 HK$ {net:,.0f}）")
    print(f"半年滾動報酬 | 最差：{w6:.2%} 最佳：{b6:.2%}")
    if trades:
        print("所有買入賣出的詳細交易記錄：")
        for t in trades:
            print(t)

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SPLK']: YFTzMissingError('possibly delisted; no timezone found')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SGEN']: YFTzMissingError('possibly delisted; no timezone found')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ATVI']: YFTzMissingError('possibly delisted; no timezone found')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ANSS']: YFTzMissingError('possibly delisted; no timezone found')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['CTXS']: YFTzMissingError('possibly delisted; no timezone found')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['CERN']: YFTzMissingError('possibly delisted; no timezone found')


總買入訊號數量（所有股票所有日子）：8508
日期 2020-04-14 00:00:00: 買入候選人數量 4
買入 TSLA 于 2020-04-14 00:00:00，數量 192，成本 9086.59
買入 AMZN 于 2020-04-14 00:00:00，數量 41，成本 4680.81
買入 NFLX 于 2020-04-14 00:00:00，數量 5，成本 2067.75
買入 ASML 于 2020-04-14 00:00:00，數量 4，成本 1096.88
日期 2020-04-15 00:00:00: 買入候選人數量 2
買入 MRNA 于 2020-04-15 00:00:00，數量 20，成本 745.00
買入 JD 于 2020-04-15 00:00:00，數量 10，成本 404.70
日期 2020-04-16 00:00:00: 買入候選人數量 4
買入 AMD 于 2020-04-16 00:00:00，數量 3，成本 170.85
日期 2020-04-17 00:00:00: 買入候選人數量 9
日期 2020-04-20 00:00:00: 買入候選人數量 5
日期 2020-04-22 00:00:00: 買入候選人數量 2
買入 SHOP 于 2020-04-22 00:00:00，數量 78，成本 4887.17
買入 PYPL 于 2020-04-22 00:00:00，數量 24，成本 2764.56
日期 2020-04-23 00:00:00: 買入候選人數量 2
買入 ZM 于 2020-04-23 00:00:00，數量 8，成本 1352.72
買入 ODFL 于 2020-04-23 00:00:00，數量 11，成本 758.26
日期 2020-04-24 00:00:00: 買入候選人數量 3
日期 2020-04-27 00:00:00: 買入候選人數量 0
日期 2020-04-28 00:00:00: 買入候選人數量 3
買入 ENPH 于 2020-04-28 00:00:00，數量 40，成本 1720.00
買入 EBAY 于 2020-04-28 00:00:00，數量 33，成本 1174.54
日期 2020-04-29 00:00:00: 買入候選人數量 5
日期 2